# Qdrant

In [1]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance

from langchain_ollama import OllamaEmbeddings

from langchain_community.document_loaders import CSVLoader

embeddings = OllamaEmbeddings(model="nomic-embed-text", base_url="http://localhost:11434")

# client = QdrantClient(":memory:")
client = QdrantClient("http://localhost:6333")
collection_name = "mlb_teams_2012-0"
if not client.collection_exists(collection_name):
    client.create_collection(collection_name, vectors_config=VectorParams(size=768, distance=Distance.COSINE))
vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embeddings,
)


def load_data():
    file_path = "./data/mlb_teams_2012.csv"
    loader = CSVLoader(file_path,
                       source_column="Team",
                       metadata_columns=("Team", "Payroll (millions)", "Wins"),
                       content_columns=("Team", "Payroll (millions)", "Wins"),
                       )
    return loader.load()


documents = load_data()
ids = vector_store.add_documents(documents)
print(ids)

['210e5d611f8a446a8c52d37621cce8be', '310ff8f4772541f2ac178020f63ebdc9', 'fd93f092263f4d8e9c8971233b5f2e81', '786cdec59d56461285c5209aea0a750a', '7032bf23a05c48e5806eea9ab1b0c93e', 'daa24d6a58d24e5c919862fa85cee988', 'ebc7502a5d044e85ba9539e18d351936', '72fb39a5211548838639bdbb36f82da7', '0ed1c87ef4274d86b4d969c09ac1816a', '94972c766bd54a7e89caaf1e166748e7', '9f8598bbf0cc49c0a9acc122611cfc5b', 'c1efa75d7af04dae978ae71e188ec2f1', '9178aeda8b7141bcb512d1ea9d3b988b', '3d20b1b879f24ea683b2705f8087ae9a', '23052ce8f5bd41c88e391f08d47c76c9', 'dbac6374b05545368de22150d04939a0', '74668385ef0446908bd0b42b4c7805d2', '1dfdd98d73ef4099bfb66d04059c8607', '80190fd4fdd94a4ab83fd7023ecaebae', '1f76238f624047ceb899042bcd0986c9', 'af6e46c72fd944ffa8f7f90b56f736d5', 'dd434eedd6a04ac69956df4e3ef6bd4f', 'ce2ab0458573491aba12d5d522492b02', '59664876ba7f4de8ba937a5c58d73bb8', 'a3d78f2e55a14e019dc15802398c16ce', '52157bd86a4143c781b510da6c621d16', '1fbe602972c340daa48b803496406d4d', '5bc07dca364c4a93bd529b1751

In [3]:
from pprint import pprint
# client = QdrantClient("http://localhost:6333")
# client = QdrantClient(host="localhost", port=6333)
client = QdrantClient(host="localhost", grpc_port=6334)
collection_info = client.get_collection(collection_name)
pprint(collection_info)

# ids = vector_store.add_documents(documents)
# collection_info = client.get_collection(collection_name)
# print(collection_info)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=30, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quan

In [2]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

res = vector_store.similarity_search("",
                                     k=100,
                                     filter=Filter(
                                         must=[FieldCondition(key="metadata.Team", match=MatchValue(value="Rockies"))],
                                     )
                                     )
print(len(res), res, sep="\n")

1
[Document(metadata={'Payroll (millions)': '       78.06', 'Team': 'Rockies', 'Wins': ' 64', 'row': 27, 'source': 'Rockies', '_id': '5bc07dca-364c-4a93-bd52-9b1751483723', '_collection_name': 'mlb_teams_2012-0'}, page_content='Team: Rockies\nPayroll (millions): 78.06\nWins: 64')]
